In [ ]:
import os
import random
from PIL import Image

import numpy as np

import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import torch.optim as optim


# FCN

Avui farem feina amb xarxes que no tenen cap tipus de capa _fully connected_ per tant serà una xarxa _Fully Convolutional Network_ (FCN). 

Emprarem un dataset classic de classificació d'imatges on s'ha de decidir si a una imatge apareix un ca o un moix. Enllaç al [Dataset](https://www.kaggle.com/c/dogs-vs-cats). 

Emprar aquest dataset ens permetrà veure com hem de procedir si no podem descarregar les dades de manera automàtica emprant la mateixa llibreria.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%ls
%cd #TODO al vostre sistema de fitxers


gdrive/  sample_data/
/content/gdrive/MyDrive/gatigos


Descomprimim el conjunt d'entrenament, serà suficient per la feina que hem de fer:


In [ ]:
#!unzip train.zip
# !unzip test1.zip # aquest avui no serà necessari

Archive:  train.zip
replace train/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Dades
Preparam el conjunt de dades, construim una llista amb el nom de totes les imatges

In [ ]:
img_files = os.listdir('train/')
img_files = list(filter(lambda x: x != 'train', img_files))
img_files = list(map(lambda p: f"train/{p}", img_files))

print("total training images", len(img_files))


random.shuffle(img_files)
#TODO: Aquí podeu substituir les X per nombres, d'aquesta manera podeu fer conjunts més grans o més petits.
# per entrenar més ràpid o més consistent
ttrain = img_files[:XXXXXX] 
ttest = img_files[XXXXXX:]

print("train size:", len(ttrain))
print("test size:", len(ttest))

Construim el dataset tal com ho hem fet sempre, però ara tota la feina és nostra

In [ ]:
# image normalization
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Constructor del dataset.
class CatDogDataset(Dataset):
    def __init__(self, image_paths, transform):
      super().__init__()
      self.paths = image_paths
      self.len = len(self.paths)
      self.transform = transform

    def __len__(self): 
      return self.len

    def __getitem__(self, index):
      path = self.paths[index]
      image = Image.open(path).convert('RGB')
      image = self.transform(image)
      label = 0 if 'cat' in path else 1
      return (image, label)


# creació dels conjunts d'entrenament i test
train_ds = CatDogDataset(ttrain, transform)
test_ds = CatDogDataset(ttest, transform)

train_dl = DataLoader(train_ds, batch_size=100)
test_dl = DataLoader(test_ds, batch_size=100)



#batch = next(iter(test_ds))
#batch[0].shape

## Xarxa
Com sempre, vosaltres us encarregau de dissenyar la xarxa:

In [ ]:
class CatAndDogConvNet(nn.Module):

    def __init__(self):
        super(CatAndDogConvNet, self).__init__()
        
        #TODO

        self.sigmoide = nn.Sigmoid()
       

    def forward(self, x):
        
        # TODO
        
        x = self.sigmoide(x)
        
        return x.squeeze()

# Entrenament

In [ ]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    
    model.train()

    loss_v = 0

    for batch_idx, (data, target) in enumerate(train_loader):
    
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
      
        loss = # TODO
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Average: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), loss.item()/ len(data)))
        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(loss_v))
 
    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            
            test_loss += #TODO
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max probability
            correct += pred.eq(target.view_as(pred)).sum().item()
 
  
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_loss

## Entrenament

In [ ]:
use_cuda = False
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

epochs = # TODO
lr = #TODO

model = CatAndDogConvNet().to(device)

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) # !!!

print("Parameters ",pytorch_total_params)
optimizer = # TODO

# Guardam el valor de peèrdua mig de cada iteració (època)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle d'entrenament
for epoch in range(0, epochs):
    train_l[epoch] = train(model, device, train_dl, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_dl)


## Validació